<a href="https://colab.research.google.com/github/gaurinotgauri/Fantasy-IPL-Prediction-Contest/blob/main/Gauri_Thakur_Fantasy_IPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#loading libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error,accuracy_score, classification_report

In [ ]:
#loading datasets
dft = pd.read_csv('/content/train_data_Total_FP.csv')
dftt = pd.read_csv('/content/test_data_Total_FP (1).csv')
dfc = pd.read_csv('/content/train_data_Captain.csv')
dfct = pd.read_csv('/content/test_data_Captain (1).csv')
dist = pd.read_csv('/content/distribution.csv')
sub = pd.read_csv('/content/sample_submission.csv')

###Analyzing data

In [ ]:
dft.shape, dfc.shape, dfct.shape, dfct.shape

((21616, 27), (21616, 27), (709, 26), (709, 26))

In [ ]:
#to check if both prediction and classification datatsets are same
def check_columns_equality(df1, df2):
    return df1.iloc[:, :26].equals(df2.iloc[:, :26])

#prediction train and test
print("The first 26 columns of dft and dfc are exactly the same." if check_columns_equality(dft, dfc) else "The first 26 columns of dft and dfc are not the same.")

#classification train and test
print("The first 26 columns of dftt and dfct are exactly the same." if check_columns_equality(dftt, dfct) else "The first 26 columns of dftt and dfct are not the same.")


The first 26 columns of dft and dfc are exactly the same.
The first 26 columns of dftt and dfct are exactly the same.


##Prediction Total_FP

Analysis of datasets

In [ ]:
#checking data types and filling of columns
dft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21616 entries, 0 to 21615
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   match_id               21616 non-null  int64  
 1   season                 21616 non-null  int64  
 2   match_name             21616 non-null  object 
 3   home_team              21616 non-null  object 
 4   away_team              21616 non-null  object 
 5   venue                  21616 non-null  object 
 6   batting_innings        21616 non-null  float64
 7   bowling_innings        21616 non-null  float64
 8   Player_name            21616 non-null  object 
 9   Starting_11            21616 non-null  float64
 10  bowling_team_bowl      11716 non-null  object 
 11  batting_team_bat       15154 non-null  object 
 12  prev_runs              14699 non-null  float64
 13  prev_balls             14699 non-null  float64
 14  prev_sixes             14699 non-null  float64
 15  pr

In [ ]:
#checking data types and filling of columns
dftt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709 entries, 0 to 708
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   match_id               709 non-null    int64  
 1   season                 709 non-null    int64  
 2   match_name             709 non-null    object 
 3   home_team              709 non-null    object 
 4   away_team              709 non-null    object 
 5   venue                  709 non-null    object 
 6   batting_innings        709 non-null    float64
 7   bowling_innings        709 non-null    float64
 8   Player_name            709 non-null    object 
 9   Starting_11            709 non-null    float64
 10  bowling_team_bowl      374 non-null    object 
 11  batting_team_bat       527 non-null    object 
 12  prev_runs              503 non-null    float64
 13  prev_balls             503 non-null    float64
 14  prev_sixes             503 non-null    float64
 15  prev_f

In [ ]:
#prediction train dataset top 5 rows
dft.head()

,match_id,season,match_name,home_team,away_team,venue,batting_innings,bowling_innings,Player_name,Starting_11,...,prev_conceded,prev_Overs_Bowled,prev_catches,prev_Dream Team,prev_Total_FP,prev_overs,prev_fielding_heroics,prev_duck,luck,Total_FP
0,1,2008,CSK v DC,CSK,DC,"MA Chidambaram Stadium, Chepauk, Chennai",2.0,1.0,AB de Villiers,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,98,32.0
1,2,2008,MI v DC,MI,DC,"Dr DY Patil Sports Academy, Mumbai",2.0,1.0,AB de Villiers,4.0,...,NaN,NaN,NaN,1.0,32.0,NaN,NaN,0,35,37.0
2,3,2008,DC v CSK,DC,CSK,"Arun Jaitley Stadium, Delhi",1.0,2.0,AB de Villiers,4.0,...,NaN,NaN,1.0,0.0,37.0,NaN,0.0,0,79,18.0
3,4,2008,RR v DC,RR,DC,"Sawai Mansingh Stadium, Jaipur",1.0,2.0,AB de Villiers,4.0,...,NaN,NaN,2.0,0.0,18.0,NaN,0.0,1,33,36.0
4,5,2008,KKR v DC,KKR,DC,"Eden Gardens, Kolkata",2.0,1.0,AB de Villiers,4.0,...,NaN,NaN,0.0,1.0,36.0,NaN,1.0,0,82,13.0


In [ ]:
#train set column names
dft.columns

Index(['match_id', 'season', 'match_name', 'home_team', 'away_team', 'venue',
       'batting_innings', 'bowling_innings', 'Player_name', 'Starting_11',
       'bowling_team_bowl', 'batting_team_bat', 'prev_runs', 'prev_balls',
       'prev_sixes', 'prev_fours', 'prev_wickets', 'prev_conceded',
       'prev_Overs_Bowled', 'prev_catches', 'prev_Dream Team', 'prev_Total_FP',
       'prev_overs', 'prev_fielding_heroics', 'prev_duck', 'luck', 'Total_FP'],
      dtype='object')

In [ ]:
#train set descriptive statistics
dft.describe()

,match_id,season,batting_innings,bowling_innings,Starting_11,prev_runs,prev_balls,prev_sixes,prev_fours,prev_wickets,prev_conceded,prev_catches,prev_Dream Team,prev_Total_FP,prev_overs,prev_fielding_heroics,prev_duck,luck,Total_FP
count,21616.000000,21616.000000,21616.000000,21616.000000,21616.0,14699.000000,14699.000000,14699.000000,14699.000000,11335.000000,11335.000000,6682.000000,20939.000000,20939.000000,11335.000000,6682.000000,21616.000000,21616.000000,21616.000000
mean,11196.635918,2015.633975,1.497641,1.502359,4.0,19.622559,15.126267,0.759848,1.776720,0.916542,25.932510,1.022897,0.510292,39.281389,3.236568,0.275367,0.071521,49.633142,37.617274
std,6452.702186,4.663043,0.500006,0.500006,0.0,21.306417,13.636834,1.343462,2.307336,0.998369,10.675894,0.628788,0.499906,33.193384,1.017250,0.519249,0.257699,28.865971,31.775817
min,1.000000,2008.000000,1.000000,1.000000,4.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-4.000000,0.000000,0.000000,0.000000,0.000000,-4.000000
25%,5611.750000,2012.000000,1.000000,1.000000,4.0,3.000000,5.000000,0.000000,0.000000,0.000000,18.500000,1.000000,0.000000,12.000000,3.000000,0.000000,0.000000,25.000000,12.000000
50%,11195.500000,2015.000000,1.000000,2.000000,4.0,12.000000,11.000000,0.000000,1.000000,1.000000,26.000000,1.000000,1.000000,31.000000,4.000000,0.000000,0.000000,50.000000,30.000000
75%,16783.250000,2020.000000,2.000000,2.000000,4.0,29.000000,22.000000,1.000000,3.000000,1.000000,33.000000,1.000000,1.000000,57.000000,4.000000,0.000000,0.000000,75.000000,55.000000
max,22362.000000,2023.000000,2.000000,2.000000,4.0,175.000000,73.000000,17.000000,19.000000,6.000000,70.000000,5.000000,1.000000,306.000000,4.000000,4.000000,1.000000,99.000000,306.000000


In [ ]:
#sum of null (NaN) values in train set
dft.isnull().sum()

match_id                     0
season                       0
match_name                   0
home_team                    0
away_team                    0
venue                        0
batting_innings              0
bowling_innings              0
Player_name                  0
Starting_11                  0
bowling_team_bowl         9900
batting_team_bat          6462
prev_runs                 6917
prev_balls                6917
prev_sixes                6917
prev_fours                6917
prev_wickets             10281
prev_conceded            10281
prev_Overs_Bowled        10281
prev_catches             14934
prev_Dream Team            677
prev_Total_FP              677
prev_overs               10281
prev_fielding_heroics    14934
prev_duck                    0
luck                         0
Total_FP                     0
dtype: int64

In [ ]:
#sum of null (NaN) values in test set
dftt.isnull().sum()

match_id                   0
season                     0
match_name                 0
home_team                  0
away_team                  0
venue                      0
batting_innings            0
bowling_innings            0
Player_name                0
Starting_11                0
bowling_team_bowl        335
batting_team_bat         182
prev_runs                206
prev_balls               206
prev_sixes               206
prev_fours               206
prev_wickets             355
prev_conceded            355
prev_Overs_Bowled        355
prev_catches             468
prev_Dream Team           28
prev_Total_FP             28
prev_overs               355
prev_fielding_heroics    468
prev_duck                  0
luck                       0
dtype: int64

In [ ]:
#unique values in train set for object type variables
unique_counts=dft.select_dtypes(include=['object']).nunique()
unique_counts

match_name            145
home_team              14
away_team              14
venue                  37
Player_name           702
bowling_team_bowl      14
batting_team_bat       14
prev_Overs_Bowled    2163
dtype: int64

In [ ]:
#unique values in prev_Overs_Bowled (also checking object type)
dft['prev_Overs_Bowled'].unique()

array([nan, '[11, 17]', '[14, 16]', ..., '[1, 6, 13, 19]',
       '[3, 12, 14, 17]', '[1, 10, 15, 17]'], dtype=object)

In [ ]:
#unique values in Starting_11
dft['Starting_11'].nunique()

1

In [ ]:
#checking if any new values are there is test set which were not in train set

#in venue column
new_venues = set(dftt['venue']).difference(set(dft['venue']))

#in the Player_name column
new_players = set(dftt['Player_name']).difference(set(dft['Player_name']))

if new_venues:
    print("New venues found:", new_venues)
else:
    print("No new venues found.")

if new_players:
    print("New players found:", new_players)
else:
    print("No new players found.")


No new venues found.
New players found: {'Kedar Devdhar', 'Mohammad Ashraful', 'Doug Bracewell'}


In [ ]:
#checking correlation between 'batting_innings' and 'bowling_innings'
correlation = dft[['batting_innings', 'bowling_innings']].corr()
print(correlation)

                 batting_innings  bowling_innings
batting_innings              1.0             -1.0
bowling_innings             -1.0              1.0


In [ ]:
#dropping columns with high null values and unnecessary inforamtion
drop_from_pred = ['match_id','Starting_11','match_name','bowling_team_bowl', 'batting_team_bat', 'prev_wickets', 'prev_conceded',
                   'prev_catches','prev_Overs_Bowled','prev_fielding_heroics','prev_overs','bowling_innings','batting_innings']
dft.drop(drop_from_pred, axis=1, inplace=True)
dftt.drop(drop_from_pred, axis=1, inplace=True)

###Filling NaN values with appropriate values

In [ ]:
#filling NaN values in 'prev_runs', 'prev_sixes', 'prev_fours', 'prev_balls' by their average values grouped by season
#joining train and test sets for better average values
dft_concat = pd.concat([dft, dftt], ignore_index=True)

#calculating average values for 'prev_runs', 'prev_sixes', 'prev_fours', 'prev_balls' grouped by season
for column in ['prev_runs', 'prev_sixes', 'prev_fours','prev_balls']:
    avg_values = dft_concat.groupby('season')[column].transform('mean')

#filling for train set
    dft[column] = dft[column].fillna(avg_values)

#filling for test set
    dftt[column] = dftt[column].fillna(avg_values)


In [ ]:
#filling NaN values in the 'prev_Dream Team' column as 0 in concatenated df
dft_concat['prev_Dream Team'] = dft_concat['prev_Dream Team'].fillna(0)
#calculating mode of prev_Dream Team dor each player
mode_dt_dft = dft_concat.groupby('Player_name')['prev_Dream Team'].apply(lambda x: x.mode().iloc[0])
#filling the mode values
dft['prev_Dream Team'] = dft['prev_Dream Team'].fillna(dft['Player_name'].map(mode_dt_dft))
dftt['prev_Dream Team'] = dftt['prev_Dream Team'].fillna(dftt['Player_name'].map(mode_dt_dft))

In [ ]:
#calculating the mean 'prev_Total_FP' value for each unique player
fp_mean_player_dft = dft_concat.groupby('Player_name')['prev_Total_FP'].mean()

#filling column with the mean 'prev_Total_FP' value for each player where NaN
dft['prev_Total_FP'] = dft['prev_Total_FP'].fillna(dft['Player_name'].map(fp_mean_player_dft))
dftt['prev_Total_FP'] = dftt['prev_Total_FP'].fillna(dftt['Player_name'].map(fp_mean_player_dft))

#filling 'prev_Total_FP' with 0 where NaN values are left
dft['prev_Total_FP'] = dft['prev_Total_FP'].fillna(0)
dftt['prev_Total_FP'] = dftt['prev_Total_FP'].fillna(0)

In [ ]:
#combining 'Player_name' column from both datasets as some players in test set are missing from train set
combined_players = pd.concat([dft['Player_name'], dftt['Player_name']], axis=0)

#LabelEncoder initializing
label_encoder = LabelEncoder()
#fitting LabelEncoder on the combined 'Player_name' column
label_encoder.fit(combined_players)

#transforming 'Player_name' column of both datasets using the fitted LabelEncoder
dft['Player_name_en'] = label_encoder.transform(dft['Player_name'])
dftt['Player_name_en'] = label_encoder.transform(dftt['Player_name'])

In [ ]:
#dropping original Player_name column
dft.drop('Player_name', axis=1, inplace=True)
dftt.drop('Player_name', axis=1, inplace=True)

In [ ]:
#One hot encoding 'venue','home_team','away_team'
dft = pd.get_dummies(dft, columns=['venue','home_team','away_team'], drop_first=True)
dftt = pd.get_dummies(dftt, columns=['venue','home_team','away_team'], drop_first=True)

###Feature engineering

In [ ]:
# new feature 'strike_rate' which is prev_runs per prev_balls
dft['strike_rate'] = dft['prev_runs'] / dft['prev_balls']
dft['strike_rate'] = dft['strike_rate'].fillna(0)
dftt['strike_rate'] = dftt['prev_runs'] / dft['prev_balls']
dftt['strike_rate'] = dftt['strike_rate'].fillna(0)

#new feature 'boundary_percentage' which is ((prev_sixes + prev_fours) per prev_runs)*100
dft['boundary_percentage'] = ((dft['prev_sixes'] + dft['prev_fours']) / dft['prev_runs']) * 100
dft['boundary_percentage'] = dft['boundary_percentage'].fillna(0)
dftt['boundary_percentage'] = ((dftt['prev_sixes'] + dftt['prev_fours']) / dftt['prev_runs']) * 100
dftt['boundary_percentage'] = dftt['boundary_percentage'].fillna(0)

In [ ]:
#replacing large values in test set with 1 dues to erroneous values from new features
large_value_threshold = 1e10
dftt.replace([np.inf, -np.inf], 1, inplace=True)
dftt[dftt > large_value_threshold] = 1

###Modelling

In [ ]:
#dividing features and target
X = dft.drop(columns=['Total_FP'])
y = dft['Total_FP']

In [ ]:
#train and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Random forest regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42) #initialize
rf_model.fit(X_train, y_train) #fit data
y_pred = rf_model.predict(X_test) #predict on test set (splitted from train set only)

rmse = np.sqrt(mean_squared_error(y_test, y_pred)) #rmse metric calculation
print("Root Mean Squared Error (RMSE):", rmse)

Root Mean Squared Error (RMSE): 32.3219243707387


In [ ]:
#prediction on actual test set
fp_pred = rf_model.predict(dftt)
FP = pd.DataFrame({'Total_FP': fp_pred})

In [ ]:
#predictions of Total_FP on test set
FP.head()

,Total_FP
0,56.62
1,37.48
2,45.63
3,45.51
4,35.19


##Classification Captain/Vice Captain

In [ ]:
#dropping unnecessary columns or columns with high NaN values
dfc_drop = ['match_id','venue','prev_runs', 'prev_balls', 'prev_sixes', 'prev_fours','luck','Starting_11','match_name','bowling_team_bowl', 'batting_team_bat', 'prev_wickets', 'prev_conceded',
                   'prev_catches','prev_Overs_Bowled','prev_fielding_heroics','prev_duck','prev_overs','batting_innings','bowling_innings','home_team','away_team']
dfc.drop(dfc_drop, axis=1, inplace=True)
dfct.drop(dfc_drop, axis=1, inplace=True)


In [ ]:
#joining train and test sets
dfc_concat = pd.concat([dfc, dfct], ignore_index=True)

In [ ]:
#filling NaN values in the 'prev_Dream Team' column as 0 in concatenated df
dfc_concat['prev_Dream Team'] = dfc_concat['prev_Dream Team'].fillna(0)
#calculating mode of prev_Dream Team dor each player
mode_dt_dfc = dfc_concat.groupby('Player_name')['prev_Dream Team'].apply(lambda x: x.mode().iloc[0])
#filling the mode values
dfc['prev_Dream Team'] = dfc['prev_Dream Team'].fillna(dfc['Player_name'].map(mode_dt_dfc))
dfct['prev_Dream Team'] = dfct['prev_Dream Team'].fillna(dfct['Player_name'].map(mode_dt_dfc))

In [ ]:
#calculating the mean 'prev_Total_FP' value for each unique player
fp_mean_player_dfc = dfc_concat.groupby('Player_name')['prev_Total_FP'].mean()

#filling with the mean 'prev_Total_FP' value for each player
dfc['prev_Total_FP'] = dfc['prev_Total_FP'].fillna(dfc['Player_name'].map(fp_mean_player_dfc))
dfct['prev_Total_FP'] = dfct['prev_Total_FP'].fillna(dfct['Player_name'].map(fp_mean_player_dfc))

#filling 'prev_Total_FP' with 0 where NaN values are left
dfc['prev_Total_FP'] = dfc['prev_Total_FP'].fillna(0)
dfct['prev_Total_FP'] = dfct['prev_Total_FP'].fillna(0)

In [ ]:
#combining 'Player_name' column from both datasets as some players in test set are missing from train set
combined_players_dfc = pd.concat([dfc['Player_name'], dfct['Player_name']], axis=0)

#LabelEncoder initializing
label_encoder = LabelEncoder()
#fitting LabelEncoder on the combined 'Player_name' column
label_encoder.fit(combined_players_dfc)

#transforming 'Player_name' column of both datasets using the fitted LabelEncoder
dfc['Player_name_en'] = label_encoder.transform(dfc['Player_name'])
dfct['Player_name_en'] = label_encoder.transform(dfct['Player_name'])


In [ ]:
#dropping original Player_name column
dfc.drop('Player_name', axis=1, inplace=True)
dfct.drop('Player_name', axis=1, inplace=True)

In [ ]:
#encoding Captain/Vice Captain
dfc['Captain/Vice Captain'] = label_encoder.fit_transform(dfc['Captain/Vice Captain'])

###Modelling

In [ ]:
#dividing features and target
X = dfc.drop(columns=['Captain/Vice Captain'])
y = dfc['Captain/Vice Captain']

In [ ]:
#train and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#XGBoost classifier
xgb_clf = XGBClassifier() #initializing
xgb_clf.fit(X_train, y_train) #fit on train data
y_pred = xgb_clf.predict(X_test) #predict on test set (splitted from train set only)

accuracy = accuracy_score(y_test, y_pred) #accuracy metric
print("Accuracy:", accuracy)

Accuracy: 0.9176688251618872


In [ ]:
#predictions of Captain/Vice Captain on actual test set
cvc_pred = xgb_clf.predict(dfct)
CVC = pd.DataFrame({'Captain/Vice Captain': cvc_pred})
#decoding Captain/Vice Captain values
original_labels = label_encoder.inverse_transform(cvc_pred)
CVC['Captain/Vice Captain'] = original_labels
#CVC.head()

In [ ]:
#making submission file
submission_df = pd.concat([sub['match_id'],FP, CVC], axis=1)
print(submission_df.info())

submission_df.to_csv('submission.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709 entries, 0 to 708
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   match_id              709 non-null    int64  
 1   Total_FP              709 non-null    float64
 2   Captain/Vice Captain  709 non-null    object 
dtypes: float64(1), int64(1), object(1)
memory usage: 16.7+ KB
None


In [ ]:
#end of code